### Import Libraries

In [38]:
import ccxt
import pandas as pd
import numpy as np
from scretos import API_KEY_ZION, API_SECRET_ZION
from datetime import date, datetime, timezone, tzinfo
import time, schedule
from finta import TA

In [78]:
#pip install matplotlib

In [79]:
import matplotlib.pyplot as plt

### Connecting to phemex Exchange

In [80]:
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': API_KEY_ZION,
    'secret': API_SECRET_ZION
})

In [81]:
print(phemex.fetch_balance())

{'info': {'code': '0', 'msg': '', 'data': [{'currency': 'USD', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1681000576133618509', 'walletVid': '0'}, {'currency': 'USDT', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1681000576073618994', 'walletVid': '0'}]}, 'USD': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'USDT': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'timestamp': 1681000576133, 'datetime': '2023-04-09T00:36:16.133Z', 'free': {'USD': 0.0, 'USDT': 0.0}, 'used': {'USD': 0.0, 'USDT': 0.0}, 'total': {'USD': 0.0, 'USDT': 0.0}}


### Building the Bot

Strategy:

* Determine the trend with sma20_1d
* Buy/Sell to open around the sma20_15m

In [82]:
symbol = 'ETHUSD' #symbol to trade
pos_size = 10 # position size
params = {'timeInForce':'PostOnly'}
target = 25

### Get Data

In [123]:
# FETCHING DATA FROM PHEMEX
timeframe = '15m' # what is the fequency of the bars?
num_bars = 10000 # how many bars of data will this fetch?
bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

# BUILDING A DATAFRAME FROM THIS DATA
df = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms') # change from milisecond timestamp

In [124]:
df

,timestamp,open,high,low,close,volume
0,2023-03-30 16:00:00,1788.26,1788.26,1780.26,1784.26,1462430.0
1,2023-03-30 16:15:00,1783.09,1790.65,1777.89,1786.86,744358.0
2,2023-03-30 16:30:00,1786.77,1788.76,1785.70,1786.88,589455.0
3,2023-03-30 16:45:00,1786.28,1786.66,1784.02,1784.74,738144.0
4,2023-03-30 17:00:00,1784.75,1786.71,1784.19,1784.49,628387.0
...,...,...,...,...,...,...
995,2023-04-10 00:45:00,1861.88,1862.11,1859.55,1859.86,174152.0
996,2023-04-10 01:00:00,1860.13,1860.47,1854.32,1854.32,459758.0
997,2023-04-10 01:15:00,1853.40,1856.03,1847.60,1849.33,539717.0
998,2023-04-10 01:30:00,1847.07,1851.06,1846.72,1850.55,593476.0


In [125]:
df['EMA200'] = df['close'].ewm(span=200, min_periods=200).mean()
df['EMA50'] = df['close'].ewm(span=50, min_periods=50).mean()
df['ATR'] = TA.ATR(df)
df['MACD'] = df['close'].ewm(span=12, min_periods=12).mean() - df['close'].ewm(span=26, min_periods=26).mean() 

In [126]:
df

,timestamp,open,high,low,close,volume,EMA200,EMA50,ATR,MACD
0,2023-03-30 16:00:00,1788.26,1788.26,1780.26,1784.26,1462430.0,NaN,NaN,NaN,NaN
1,2023-03-30 16:15:00,1783.09,1790.65,1777.89,1786.86,744358.0,NaN,NaN,NaN,NaN
2,2023-03-30 16:30:00,1786.77,1788.76,1785.70,1786.88,589455.0,NaN,NaN,NaN,NaN
3,2023-03-30 16:45:00,1786.28,1786.66,1784.02,1784.74,738144.0,NaN,NaN,NaN,NaN
4,2023-03-30 17:00:00,1784.75,1786.71,1784.19,1784.49,628387.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,2023-04-10 00:45:00,1861.88,1862.11,1859.55,1859.86,174152.0,1854.444323,1853.787941,4.752857,3.087094
996,2023-04-10 01:00:00,1860.13,1860.47,1854.32,1854.32,459758.0,1854.443086,1853.808807,4.805000,2.285903
997,2023-04-10 01:15:00,1853.40,1856.03,1847.60,1849.33,539717.0,1854.392207,1853.633167,4.781429,1.234077
998,2023-04-10 01:30:00,1847.07,1851.06,1846.72,1850.55,593476.0,1854.353975,1853.512259,4.466429,0.493254


### Conditions

In [128]:
c1 = df.close > df.EMA200
c2 = (df.low < df.EMA50) & (df.close > df.EMA50) & (df.open > df.EMA50)
long_entry_condition = c1 & c2

In [145]:
df.loc[df.loc[long_entry_condition].index, 'sig'] = 'BUY'

In [149]:
df

,timestamp,open,high,low,close,volume,EMA200,EMA50,ATR,MACD,sig
0,2023-03-30 16:00:00,1788.26,1788.26,1780.26,1784.26,1462430.0,NaN,NaN,NaN,NaN,NaN
1,2023-03-30 16:15:00,1783.09,1790.65,1777.89,1786.86,744358.0,NaN,NaN,NaN,NaN,NaN
2,2023-03-30 16:30:00,1786.77,1788.76,1785.70,1786.88,589455.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-30 16:45:00,1786.28,1786.66,1784.02,1784.74,738144.0,NaN,NaN,NaN,NaN,NaN
4,2023-03-30 17:00:00,1784.75,1786.71,1784.19,1784.49,628387.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,2023-04-10 00:45:00,1861.88,1862.11,1859.55,1859.86,174152.0,1854.444323,1853.787941,4.752857,3.087094,NaN
996,2023-04-10 01:00:00,1860.13,1860.47,1854.32,1854.32,459758.0,1854.443086,1853.808807,4.805000,2.285903,NaN
997,2023-04-10 01:15:00,1853.40,1856.03,1847.60,1849.33,539717.0,1854.392207,1853.633167,4.781429,1.234077,NaN
998,2023-04-10 01:30:00,1847.07,1851.06,1846.72,1850.55,593476.0,1854.353975,1853.512259,4.466429,0.493254,NaN


### Moving Averages

    'timeframes': {                      // empty if the exchange.has['fetchOHLCV'] !== true
        '1m': '1minute',
        '1h': '1hour',
        '1d': '1day',
        '1M': '1month',
        '1y': '1year',
    }

### Determine the Trend

* if bid < sma20_1d then BEARISH
* if bid > sma20_1d then BULLISH

```BUY/SELL TO OPEN AROUND THE SMA20_15m - .1% UNDER AND .3% OVER```

In [130]:
def ask_bid():
    """" ask_bid()[0] == ask"""
    ob = phemex.fetch_order_book(symbol=symbol) # returns the orderbook object wich is a dictionary
    bid = ob['bids'][0][0]
    ask = ob['asks'][0][0]
    return ask, bid

In [10]:
#  FIND DAILY SMA 20
def sma20_1d():

    print('####################### Starting Daily SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '1d' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_1d = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_1d['timestamp'] = pd.to_datetime(df_1d['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20d
    df_1d['sma20_1d'] = df_1d['close'].rolling(20).mean()

    # if bid < sma20_1d then BEARISH, if bid > sma20_1d then BULLISH
    bid = ask_bid()[1]
    print(f'##################bid:{bid}####################')
    df_1d.loc[df_1d['sma20_1d']>bid, 'sig'] = 'SELL'
    df_1d.loc[df_1d['sma20_1d']<bid, 'sig'] = 'BUY'
 
    return df_1d

In [11]:
sma20_1d()

####################### Starting Daily SMA #######################
##################bid:28419.9####################


,timestamp,open,high,low,close,volume,sma20_1d,sig
0,2022-06-13,26530.5,26843.5,21846.5,22459.5,21730512.0,NaN,NaN
1,2022-06-14,22450.5,24453.5,20743.5,22102.5,15032941.0,NaN,NaN
2,2022-06-15,22111.0,22848.0,20000.0,22570.0,16385422.0,NaN,NaN
3,2022-06-16,22570.0,22998.0,20200.5,20370.5,14348298.0,NaN,NaN
4,2022-06-17,20371.0,21396.0,20210.0,20449.0,15372249.0,NaN,NaN
...,...,...,...,...,...,...,...,...
296,2023-04-05,28165.4,28779.9,27809.6,28174.3,15786531.0,27841.545,BUY
297,2023-04-06,28168.3,28174.8,27686.9,28032.8,21799146.0,27871.115,BUY
298,2023-04-07,28030.4,28114.7,27780.0,27914.3,20779309.0,27918.035,BUY
299,2023-04-08,27914.2,28155.7,27873.1,27941.0,12145490.0,27913.475,BUY


Continue with Strategy...

In [12]:
#  FIND 15MINUTE SMA, FIGURE OUT IF BUY OR SELL BASED ON bid vs sma20_1d, if bid < sma20_1d then SELL
def sma20_15m():

    print('####################### Starting 15m SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '15m' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_15m = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_15m['timestamp'] = pd.to_datetime(df_15m['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20_15m
    df_15m['sma20_15m'] = df_15m['close'].rolling(20).mean()

    # BUY PRICE 1+2 and SELL PRICE 1+2
    df_15m['bp_1'] = df_15m['sma20_15m'] * 1.001 # sma20_15m 0.1% under and 0.3%over 
    df_15m['bp_2'] = df_15m['sma20_15m'] * 0.997
    df_15m['sp_1'] = df_15m['sma20_15m'] * 0.999
    df_15m['sp_2'] = df_15m['sma20_15m'] * 1.003
    
    return df_15m

In [13]:
sma20_15m()

####################### Starting 15m SMA #######################


,timestamp,open,high,low,close,volume,sma20_15m,bp_1,bp_2,sp_1,sp_2
0,2023-03-30 14:45:00,28426.6,28436.2,28373.8,28413.9,334470.0,NaN,NaN,NaN,NaN,NaN
1,2023-03-30 15:00:00,28401.5,28467.2,28375.4,28428.2,220136.0,NaN,NaN,NaN,NaN,NaN
2,2023-03-30 15:15:00,28428.1,28456.6,28336.0,28337.7,542971.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-30 15:30:00,28346.9,28379.4,28283.1,28331.8,894165.0,NaN,NaN,NaN,NaN,NaN
4,2023-03-30 15:45:00,28320.4,28340.0,28160.0,28303.3,297111.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,2023-04-09 23:30:00,28364.4,28378.0,28300.0,28300.0,18810.0,28245.405,28273.650405,28160.668785,28217.159595,28330.141215
996,2023-04-09 23:45:00,28323.3,28337.5,28304.0,28337.1,108234.0,28260.760,28289.020760,28175.977720,28232.499240,28345.542280
997,2023-04-10 00:00:00,28332.7,28380.0,28294.1,28369.2,303117.0,28274.940,28303.214940,28190.115180,28246.665060,28359.764820
998,2023-04-10 00:15:00,28366.2,28385.8,28353.7,28384.3,286082.0,28284.655,28312.939655,28199.801035,28256.370345,28369.508965


### Code the bot

In [14]:
def open_positions():
    params = {'type':'swap', 'code':'USD'}
    phe_bal = phemex.fetch_balance(params=params)
    open_positions = phe_bal['info']['data']['positions']
    open_positions_side = open_positions[0]['side']
    open_positions_size = open_positions[0]['size']

    if open_positions_side == ('Buy'):
        openpos_bool = True
        long = True
    elif open_positions_side == ('Sell'):
        openpos_bool = True
        long = False
    else:
        openpos_bool = False
        long = None
    return open_positions, openpos_bool, open_positions_size, long

In [15]:
open_positions()[0]

[{'userID': '4779771',
  'accountID': '47797710002',
  'symbol': 'uBTCUSD',
  'currency': 'USD',
  'side': 'Buy',
  'positionStatus': 'Normal',
  'crossMargin': False,
  'leverageEr': '500000000',
  'leverage': '5.00000000',
  'initMarginReqEr': '20000000',
  'initMarginReq': '0.20000000',
  'maintMarginReqEr': '500000',
  'maintMarginReq': '0.00500000',
  'riskLimitEv': '20000000000',
  'riskLimit': '2000000.00000000',
  'size': '5',
  'value': '139.54350000',
  'valueEv': '1395435',
  'avgEntryPriceEp': '279087000',
  'avgEntryPrice': '27908.70000000',
  'posCostEv': '279757',
  'posCost': '27.97570000',
  'assignedPosBalanceEv': '279757',
  'assignedPosBalance': '27.97570000',
  'bankruptCommEv': '670',
  'bankruptComm': '0.06700000',
  'bankruptPriceEp': '223270000',
  'bankruptPrice': '22327.00000000',
  'positionMarginEv': '279087',
  'positionMargin': '27.90870000',
  'liquidationPriceEp': '224666000',
  'liquidationPrice': '22466.60000000',
  'deleveragePercentileEr': '0',
  'd

In [16]:
# CREATE A KILL_SWITCH
def kill_switch():
    print('starting kill switch')
    openposi = open_positions()[1]
    kill_size = open_positions()[2]
    long = open_positions()[3]

    while openposi == True:
        print('starting kill switch loop till limit fill..')
        temp_df = pd.DataFrame()
        print("just made a temporary df")

        phemex.cancel_all_orders(symbol=symbol)
        openposi = open_positions()[1]
        kill_size = open_positions()[2]
        long = open_positions()[3]

        ask = ask_bid()[0]
        bid = ask_bid()[1]

        if long == False:
            phemex.create_limit_buy_order(symbol=symbol, amount=kill_size, price=bid, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${bid}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        elif long == True:
            phemex.create_limit_sell_order(symbol=symbol, amount=kill_size, price=ask, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${ask}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        else:
            print("++++++ Something I didn't expect in kill switch function")

        openposi = open_positions()[1]
    return

In [17]:
kill_switch()

starting kill switch
starting kill switch loop till limit fill..
just made a temporary df
just made a BUY order to close order of 5|uBTCUSD at $28419.3
sleeping for 30 seconds to see if it fills
starting kill switch loop till limit fill..
just made a temporary df
just made a BUY order to close order of 5|uBTCUSD at $28411.7
sleeping for 30 seconds to see if it fills


In [ ]:
# CREATE A PNL CLOSE
def pnl_close():

    current_price = ask_bid()[1]
    # if hit target, close
    print("checking to see if it is time to exit...")
    params = {"type":"swap", "code":"USD"}
    pos_dict = phemex.fetch_positions(params=params)
    pos_dict = pos_dict[0]
    print(pos_dict)
    side = pos_dict["side"]
    #size = pos_dict["contracts"]
    entry_price = float(pos_dict["entryPrice"])
    leverage = float(pos_dict["leverage"])

    print(f'side:{side} | entry_price:{entry_price} | leverage:{leverage}')

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        percentage = round(((diff/entry_price)*leverage), 10)
        print(percentage)
    except:
        percentage = 0
        print(percentage)

    print(f'diff:{diff} | percentage:{percentage}')
    pnl = percentage*100
    #pnl = 26
    
    print(f"this is our pnl {pnl}")

    pnlclose = False
    in_position = False

    if pnl > 0:
        in_position = True
        print("we are in a winning position")
        if pnl > target:
            print(f':) :) :) starting the kill switch becaue we hit our target:{target}')
            pnlclose = True
            kill_switch()
    elif pnl <0:
        print("we are in a loosing position but holding on")
        in_position = True
    else:
        print("we are not in a position")

    return pnlclose, in_position # pnl_close()[0] == pnlclose, pnl_close()[1] == in_position, size 

In [ ]:
pnl_close()

checking to see if it is time to exit...
{'info': {'userID': '4779771', 'accountID': '47797710002', 'symbol': 'uBTCUSD', 'currency': 'USD', 'side': 'Buy', 'positionStatus': 'Normal', 'crossMargin': False, 'leverageEr': '500000000', 'leverage': '5.00000000', 'initMarginReqEr': '20000000', 'initMarginReq': '0.20000000', 'maintMarginReqEr': '500000', 'maintMarginReq': '0.00500000', 'riskLimitEv': '20000000000', 'riskLimit': '2000000.00000000', 'size': '5', 'value': '139.54350000', 'valueEv': '1395435', 'avgEntryPriceEp': '279087000', 'avgEntryPrice': '27908.70000000', 'posCostEv': '279757', 'posCost': '27.97570000', 'assignedPosBalanceEv': '279757', 'assignedPosBalance': '27.97570000', 'bankruptCommEv': '670', 'bankruptComm': '0.06700000', 'bankruptPriceEp': '223270000', 'bankruptPrice': '22327.00000000', 'positionMarginEv': '279087', 'positionMargin': '27.90870000', 'liquidationPriceEp': '224666000', 'liquidationPrice': '22466.60000000', 'deleveragePercentileEr': '0', 'deleveragePercenti

(False, True)

In [ ]:
def bot():
    df_1d = sma20_1d()
    df_15m = sma20_15m()
    ask, bid = ask_bid()
    in_position = pnl_close()[1]

    sig = df_1d.iloc[-1]['sig']
    #sig = 'SELL'
    open_size = pos_size/2

    if in_position == False:
        if sig == "BUY":
            print('making opening order as a BUY')
            bp_1 = df_15m.iloc[-1]['bp_1']
            bp_2 = df_15m.iloc[-1]['bp_2']
            print('bp1:',bp_1)
            print('bp2:',bp_2)

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_1, params=params)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_2, params=params)

            #time.sleep(5)
        else:
            print("making opening order as a SELL")
            sp_1 = df_15m.iloc[-1]['sp_1']
            sp_2 = df_15m.iloc[-1]['sp_2']

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_1, params=params)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_2, params=params)

            #time.sleep(5)
    else:
        print("we are in position already, so not making new orders...")
    return

In [ ]:
bot()

####################### Starting Daily SMA #######################
##################bid:1863.51####################
####################### Starting 15m SMA #######################
checking to see if it is time to exit...
{'info': {'userID': '4779771', 'accountID': '47797710002', 'symbol': 'uBTCUSD', 'currency': 'USD', 'side': 'Buy', 'positionStatus': 'Normal', 'crossMargin': False, 'leverageEr': '500000000', 'leverage': '5.00000000', 'initMarginReqEr': '20000000', 'initMarginReq': '0.20000000', 'maintMarginReqEr': '500000', 'maintMarginReq': '0.00500000', 'riskLimitEv': '20000000000', 'riskLimit': '2000000.00000000', 'size': '5', 'value': '139.54350000', 'valueEv': '1395435', 'avgEntryPriceEp': '279087000', 'avgEntryPrice': '27908.70000000', 'posCostEv': '279757', 'posCost': '27.97570000', 'assignedPosBalanceEv': '279757', 'assignedPosBalance': '27.97570000', 'bankruptCommEv': '670', 'bankruptComm': '0.06700000', 'bankruptPriceEp': '223270000', 'bankruptPrice': '22327.00000000', 'pos